In [32]:
import sagemaker
import boto3
from sagemaker.amazon.amazon_estimator import get_image_uri
from sagemaker.session import s3_input,Session
from sagemaker.image_uris import get_training_image_uri

In [20]:
pip install urilib

ERROR: Could not find a version that satisfies the requirement urilib (from versions: none)
ERROR: No matching distribution found for urilib
Note: you may need to restart the kernel to use updated packages.


In [15]:
bucket_name = 'arthitapp'
region = boto3.session.Session().region_name
print(region)

ap-south-1


In [16]:
s3 = boto3.resource('s3')
try:
    if region == 'ap-south-1' :
        s3.create_bucket(Bucket=bucket_name,
                        CreateBucketConfiguration={'LocationConstraint': 'ap-south-1'})
        print("bucket created")
except Exception as e:
    print("S3 error ",e)

bucket created


In [17]:
prefix = "xg-boost-built-in-algo"
output_path = 's3://{}/{}/output'.format(bucket_name,prefix)
print(output_path)

s3://arthitapp/xg-boost-built-in-algo/output


In [5]:
import pandas as pd
import urllib
try:
    urllib.request.urlretrieve ("https://d1.awsstatic.com/tmt/build-train-deploy-machine-learning-model-sagemaker/bank_clean.27f01fbbdf43271788427f3682996ae29ceca05d.csv", "bank_clean.csv")
    print('Success: downloaded bank_clean.csv.')
except Exception as e:
    print('Data load error: ',e)

Success: downloaded bank_clean.csv.


In [11]:
import numpy as np
df = pd.read_csv('bank_clean.csv')
train_data, test_data = np.split(df.sample(frac=1, random_state=1729), [int(0.7 * len(df))])
print(train_data.shape, test_data.shape)


(28831, 62) (12357, 62)


/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/numpy/core/fromnumeric.py:57: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


SyntaxError: invalid syntax. Maybe you meant '==' or ':=' instead of '='? (3183530266.py, line 4)

In [20]:
import os
pd.concat([train_data['y_yes'], train_data.drop(['y_no', 'y_yes'], 
                                                axis=1)], 
                                                axis=1).to_csv('train.csv', index=False, header=False)
boto3.Session().resource('s3').Bucket(bucket_name).Object(os.path.join(prefix, 'train/train.csv')).upload_file('train.csv')
s3_input_train = sagemaker.TrainingInput(s3_data='s3://{}/{}/train'.format(bucket_name, prefix), content_type='csv')

In [22]:
pd.concat([test_data['y_yes'], test_data.drop(['y_no', 'y_yes'], axis=1)], axis=1).to_csv('test.csv', index=False, header=False)
boto3.Session().resource('s3').Bucket(bucket_name).Object(os.path.join(prefix, 'test/test.csv')).upload_file('test.csv')
s3_input_test = sagemaker.TrainingInput(s3_data='s3://{}/{}/test'.format(bucket_name, prefix), content_type='csv')

In [35]:
container = get_training_image_uri(boto3.Session().region_name,
                          'xgboost', 
                          framework_version='1.0-1')

In [36]:
hyperparameters = {
        "max_depth":"5",
        "eta":"0.2",
        "gamma":"4",
        "min_child_weight":"6",
        "subsample":"0.7",
        "objective":"binary:logistic",
        "num_round":50
        }

In [40]:
estimator = sagemaker.estimator.Estimator(image_uri=container, 
                                          hyperparameters=hyperparameters,
                                          role=sagemaker.get_execution_role(),
                                          instance_count=1, 
                                          instance_type='ml.m5.2xlarge', 
                                          volume_size=5, # 5 GB 
                                          output_path=output_path,
                                          use_spot_instances=True,
                                          max_run=300,
                                          max_wait=600)

In [41]:
estimator.fit({'train': s3_input_train,'validation': s3_input_test})

INFO:sagemaker:Creating training-job with name: sagemaker-xgboost-2024-03-22-11-12-03-631


2024-03-22 11:12:03 Starting - Starting the training job...
2024-03-22 11:12:18 Starting - Preparing the instances for training...
2024-03-22 11:13:04 Downloading - Downloading input data...
2024-03-22 11:13:24 Downloading - Downloading the training image...
2024-03-22 11:13:44 Training - Training image download completed. Training in progress...[2024-03-22 11:14:08.332 ip-10-0-174-143.ap-south-1.compute.internal:7 INFO utils.py:27] RULE_JOB_STOP_SIGNAL_FILENAME: None
INFO:sagemaker-containers:Imported framework sagemaker_xgboost_container.training
INFO:sagemaker-containers:Failed to parse hyperparameter objective value binary:logistic to Json.
Returning the value itself
INFO:sagemaker-containers:No GPUs detected (normal if no gpus installed)
INFO:sagemaker_xgboost_container.training:Running XGBoost Sagemaker in algorithm mode
INFO:root:Determined delimiter of CSV input is ','
INFO:root:Determined delimiter of CSV input is ','
INFO:root:Determined delimiter of CSV input is ','
INFO:roo

In [42]:
xgb_predictor = estimator.deploy(initial_instance_count=1,instance_type='ml.m4.xlarge')

INFO:sagemaker:Creating model with name: sagemaker-xgboost-2024-03-22-11-18-07-237
INFO:sagemaker:Creating endpoint-config with name sagemaker-xgboost-2024-03-22-11-18-07-237
INFO:sagemaker:Creating endpoint with name sagemaker-xgboost-2024-03-22-11-18-07-237


-------!

In [53]:
from sagemaker.serializers import CSVSerializer
import numpy as np

test_data_array = test_data.drop(['y_no', 'y_yes'], axis=1).values  # Load the data into an array

xgb_predictor.content_type = 'text/csv'  # Set the data type for an inference
xgb_predictor.serializer = CSVSerializer()  # Instantiate the CSVSerializer

serialized_test_data = xgb_predictor.serializer.serialize(test_data_array)  # Serialize the test data

predictions = xgb_predictor.predict(serialized_test_data).decode('utf-8')  # Predict!
predictions_array = np.fromstring(predictions[1:], sep=',')  # Convert the predictions to an array

print(predictions_array.shape)

(12357,)


In [54]:
predictions_array

array([0.05825063, 0.04520085, 0.0379552 , ..., 0.01467592, 0.0270674 ,
       0.03558293])

In [55]:
cm = pd.crosstab(index=test_data['y_yes'], columns=np.round(predictions_array), rownames=['Observed'], colnames=['Predicted'])
tn = cm.iloc[0,0]; fn = cm.iloc[1,0]; tp = cm.iloc[1,1]; fp = cm.iloc[0,1]; p = (tp+tn)/(tp+tn+fp+fn)*100
print("\n{0:<20}{1:<4.1f}%\n".format("Overall Classification Rate: ", p))
print("{0:<15}{1:<15}{2:>8}".format("Predicted", "No Purchase", "Purchase"))
print("Observed")
print("{0:<15}{1:<2.0f}% ({2:<}){3:>6.0f}% ({4:<})".format("No Purchase", tn/(tn+fn)*100,tn, fp/(tp+fp)*100, fp))
print("{0:<16}{1:<1.0f}% ({2:<}){3:>7.0f}% ({4:<}) \n".format("Purchase", fn/(tn+fn)*100,fn, tp/(tp+fp)*100, tp))


Overall Classification Rate: 89.9%

Predicted      No Purchase    Purchase
Observed
No Purchase    91% (10738)    34% (198)
Purchase        9% (1045)     66% (376) 

